In [18]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq

In [19]:
import os
llm = Groq(model="llama-3.3-70b-versatile", api_key=os.environ.get("GROQ_API_KEY"))

In [20]:
def calcular_imposto_renda(rendimento:float) -> str:
    """ 
    Calcula o imposto de renda com base no rendimento anual.
    Args:
        rendimento (float): Rendimento anual do contribuinte.
    Returns:
        str: O valor do imposto devido com base no rendimento.
    """
    if rendimento <= 2000:
        return "Isento de imposto de renda."
    elif 2000 < rendimento <= 5000:
        imposto = (rendimento - 2000) * 0.10
        return f"Imposto devido é de: R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}."
    elif 5000 < rendimento <= 10000:
        imposto = (rendimento - 5000) * 0.15 + 300
        return f"Imposto devido é de: R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}."
    else:
        imposto = (rendimento - 10000) * 0.20 + 1050
        return f"Imposto devido é de: R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}."

### Convertendo Função em Ferramenta

In [21]:
from llama_index.core.tools import FunctionTool

In [22]:
ferramenta_imposto_renda = FunctionTool.from_defaults(
    fn=calcular_imposto_renda,
    name="Calcular Imposto de Renda",
    description=(
    "Calcula o imposto de renda com base no rendimento anual do contribuinte."
    "Argumento: redimento (float)."
    "Retorna o valor do imposto devido de acordo com faixas de rendimento."
    )
)

In [23]:
from llama_index.core.agent import FunctionCallingAgentWorker 

In [24]:
agent_worker_imposto = FunctionCallingAgentWorker.from_tools(
    tools=[ferramenta_imposto_renda],
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

In [25]:
from llama_index.core.agent import AgentRunner

In [26]:
agent_imposto= AgentRunner(agent_worker_imposto)

In [28]:
response = agent_imposto.chat("""
    Qual é o imposto de renda devido por uma pessoa com rendimento anual de R$ 7500?
"""
)

Added user message to memory: 
    Qual é o imposto de renda devido por uma pessoa com rendimento anual de R$ 7500?

=== Calling Function ===
Calling function: Calcular Imposto de Renda with args: {"rendimento": 7500}
=== Function Output ===
Imposto devido é de: R$ 675.00, base em um rendimento de R$ 7500.00.
=== LLM Response ===
O imposto de renda devido por uma pessoa com rendimento anual de R$ 7500 é de R$ 675,00.


In [30]:
import arxiv 

def consulta_artigos(titulo: str) -> str:
    """ Consulta artigos na base de dados arXiv e retorna resultados formatados."""
    busca = arxiv.Search(
        query=titulo,
        max_results=5,
        sort_by=arxiv.SortCriterion.Relevance
    )

    resultados = [
        f"Título: {artigo.tiltle}\n"
        f"Categoria: {artigo.primary_category}\n"
        f"Link: {artigo.entry_id}\n"
        for artigo in busca.results()
    ]

    return "\n\n".join(resultados)